In [1]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
all_data = pd.read_excel('./streamlit/data/WPP2022.xlsx', skiprows=16, index_col=0)

In [9]:
countries = all_data[all_data['Type'] == 'Country/Area']

In [10]:
country_continent = pd.read_csv('./streamlit/data/country_continent.csv')

In [11]:
countries = countries.merge(country_continent[['Country_Number', 'Continent_Name', 'Three_Letter_Country_Code']], how='left', left_on='Location code', right_on='Country_Number')

In [12]:
GDP = pd.read_csv('./streamlit/data/GDP.csv', skiprows=4)
GDP = GDP.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis=1)
GDP = GDP.melt(id_vars='Country Code', value_vars=set(GDP.columns) - set('Country Code'), var_name='Year', value_name='GDP_per_capita').sort_values('Country Code')
GDP['Year'] = pd.to_numeric(GDP['Year'], errors='coerce')

In [13]:
countries = countries.merge(GDP, how='left', left_on=['Three_Letter_Country_Code', 'Year'], right_on=['Country Code', 'Year'])

In [14]:
countires = countries[countries['GDP_per_capita'].notna()]

In [15]:
columns = ['Total Population, as of 1 January (thousands)', 'Median Age, as of 1 July (years)', 'Life Expectancy at Birth, both sexes (years)', 'GDP_per_capita', 'Infant Mortality Rate (infant deaths per 1,000 live births)', 'Under-Five Deaths, under age 5 (thousands)']

In [16]:
DATA = countries[columns]
for column in columns:
    DATA[column] = pd.to_numeric(DATA[column], errors='coerce')

/tmp/ipykernel_32780/356292927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA[column] = pd.to_numeric(DATA[column], errors='coerce')


In [17]:
DATA = DATA.dropna()

In [18]:
DATA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10552 entries, 10 to 17495
Data columns (total 6 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Total Population, as of 1 January (thousands)                10552 non-null  float64
 1   Median Age, as of 1 July (years)                             10552 non-null  float64
 2   Life Expectancy at Birth, both sexes (years)                 10552 non-null  float64
 3   GDP_per_capita                                               10552 non-null  float64
 4   Infant Mortality Rate (infant deaths per 1,000 live births)  10552 non-null  float64
 5   Under-Five Deaths, under age 5 (thousands)                   10552 non-null  float64
dtypes: float64(6)
memory usage: 577.1 KB


In [19]:
X = DATA.drop('Life Expectancy at Birth, both sexes (years)', axis=1)
y = DATA['Life Expectancy at Birth, both sexes (years)']

x_train,x_test , y_train,y_test = train_test_split(X, y ,\
                                                   test_size=.2, random_state=313)


std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)

In [20]:
poly_transformer = PolynomialFeatures(degree=2)
poly_transformer.fit(x_train)
poly_features = poly_transformer.transform(x_train)

In [21]:
model = LinearRegression()
model.fit(poly_features, y_train)

LinearRegression()

In [22]:
# evaluate model
from sklearn.metrics import r2_score

y_pred = model.predict(poly_transformer.transform(std_scaler.transform(x_test)))
r2_score(y_test, y_pred)

0.9243605251394174